In [1]:
# imports
from confluent_kafka import Producer, Consumer
import json

In [2]:
# Kafka defintions
KAFKA_BROKER = "kafka:9092"
TOPIC = "sample-weather-data"

## Weather producer -> send to Kafka Topic
This notebook pushes data from an exmple file (sample_weather_data.json) to a Kafka topic for further processing.


In [3]:
import time

# Delivery callback for confirmation
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.value()}: {err}")
    else:
        print(f"Record delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Create Confluent Kafka producer
producer = Producer({'bootstrap.servers': KAFKA_BROKER})

# Load sample data
with open("./data/sample_weather_data.json", "r") as file:
    weather_data = json.load(file)

weather_data

[{'sensor_id': 'WS_001',
  'temperature': 15.2,
  'humidity': 65,
  'wind_speed': 12.4,
  'timestamp': '2025-02-14T10:00:00Z'},
 {'sensor_id': 'WS_002',
  'temperature': 18.5,
  'humidity': 55,
  'wind_speed': 10.1,
  'timestamp': '2025-02-14T10:01:00Z'},
 {'sensor_id': 'WS_003',
  'temperature': 21.0,
  'humidity': 50,
  'wind_speed': 8.7,
  'timestamp': '2025-02-14T10:02:00Z'}]

In [4]:
# Send messages to Kafka
for record in weather_data:
    producer.produce(
        TOPIC,
        value=json.dumps(record).encode('utf-8'),
        callback=delivery_report
    )
    producer.poll(0)  # Serve delivery callbacks
    time.sleep(1)  # Simulate real-time streaming

producer.flush()

Record delivered to sample-weather-data [0] at offset 0
Record delivered to sample-weather-data [0] at offset 1
Record delivered to sample-weather-data [0] at offset 2


0

## Weather consumer --> read from Kafka
Read the data from Kafka and print the results.

In [ ]:
# Create Kafka consumer configuration
conf = {
    'bootstrap.servers': KAFKA_BROKER,
    'group.id': 'weather-consumer-group',
    'auto.offset.reset': 'earliest'
}

# Create Consumer instance
consumer = Consumer(conf)

# Subscribe to topic
consumer.subscribe([TOPIC])

print("Waiting for messages...")

try:
    while True:
        msg = consumer.poll(1.0)  # timeout in seconds
        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        weather_data = json.loads(msg.value().decode('utf-8'))
        print(f"Received: {weather_data}")

except KeyboardInterrupt:
    print("Stopping consumer...")

finally:
    consumer.close()

Waiting for messages...
Received: {'sensor_id': 'WS_001', 'temperature': 15.2, 'humidity': 65, 'wind_speed': 12.4, 'timestamp': '2025-02-14T10:00:00Z'}
Received: {'sensor_id': 'WS_002', 'temperature': 18.5, 'humidity': 55, 'wind_speed': 10.1, 'timestamp': '2025-02-14T10:01:00Z'}
Received: {'sensor_id': 'WS_003', 'temperature': 21.0, 'humidity': 50, 'wind_speed': 8.7, 'timestamp': '2025-02-14T10:02:00Z'}
